In [ ]:
# first method tried
# method based on Bag of visual words and SIFT
# step 1: construct the dictionary of the visual words
# step 2: classify the images using the clusters from the dictionary (training)
# step 3: classify the images using the clusters from the dictionary (test)

# images were not rotated, wanted to test the method on normal images, the results were not promising, discussed the method
# with the supervisor, and he discouraged such complex ideas (SIFT) and wanted simpler ideas, as the presentation 
# is limited to 15 minutes and we had other problems that we had to deal with (translation, resizing)
###
# so the next one was using contouring to find the angle

# with thresholding 
# sample length:  9708
# accuracy:  53.56407086938607

In [26]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from matplotlib.pyplot import imshow
import math
import os, os.path
import webbrowser

In [53]:
# setting the number of best features to retain to 15 for each image
sift = cv2.xfeatures2d.SIFT_create(nfeatures = 15)
# sift = cv2.xfeatures2d.SIFT_create()

In [54]:
# here descriptors are found to be used to find clusters (bag of visual words)

imgs = []
path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_training"

exclude_count = 0
total_desc_samples = []

valid_images = [".jpg"]
for i in range(10):
    total_descriptors = 0
    chosen_per_folder = 0
    imgs_subset=[]
    path = path + "\\" + str(i)
    print(i)
    for f in os.listdir(path):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_images:
            continue
        image = cv2.imread(os.path.join(path,f))
        keypoints, descriptors = sift.detectAndCompute(image, None)

        if(len(keypoints) > 0):
            total_desc_samples.extend(descriptors.tolist())

    path = path.rsplit('\\',1)[0]

0
1
2
3
4
5
6
7
8
9


In [55]:
# Bag of visual words
# here we group to the descriptors to clusters, so that these clsuters will be used to classify the descriptors of the
# images (training and test) we have later


import faiss
import time

# kmeans = KMeans(n_clusters=40, random_state=0).fit(total_desc_samples)
# ncentroids = 1024
# niter = 20
# verbose = True
# d = x.shape[1]
# kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
# kmeans.train(x)

total_desc_samples = np.array(total_desc_samples)

k = 200
n_init = 10
max_iter = 300
kmeans_faiss = faiss.Kmeans(d=total_desc_samples.shape[1], k=k, niter=max_iter, nredo=n_init)
s = time.time()
print(s)
kmeans_faiss.train(total_desc_samples.astype(np.float32))
e = time.time()
print("Training time = {}".format(e - s))

1634570367.1476102
Training time = 54.478182315826416


In [31]:
# COMMENT ADDED LATER: time taken for the faiss algorithm to finish (signicantly less than the tradiditonal kmeans in our
# usecase, i think traditional kmeans took around 10 times longer)

# Training time = 5.611945152282715 (clusters = 40, max_iter = 300)
# Training time = 8.646806001663208 (clusters = 40, max_iter = 500)
# Training time = 34.214611291885376 (clusters = 100, max_iter = 500)
# Training time = 89.70109343528748 (best time circa: 80) (clusters = 200, max_iter = 500) 
# Training time = 182.177996635437 (best time= 154.5881745815277) (clusters = 300, max_iter = 500)

In [56]:
# now We check the feature descriptors (keypoints) generated from each image in the training set against
# the vocabulary to determine how many of the feature descriptors, saved in the last step, are present in each image.
# and these bin assignments are used later to classify the images


training_data = []
training_label = []
path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_training"
valid_images = [".jpg"]

for i in range(10):
    path = path + "\\" + str(i)
#     print(path)
    print(i)
    for f in os.listdir(path):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_images:
            continue
        
        # read the image
        image = cv2.imread(os.path.join(path,f))
        keypoints, descriptors = sift.detectAndCompute(image, None)
        
        # for each descriptor, find the nearest cluster
        if len(keypoints) != 0:
            bin_assignments = kmeans_faiss.index.search(descriptors.astype(np.float32), 1)[1]
            bin_assignments = bin_assignments.reshape(1, bin_assignments.shape[0])
            
            # sort, improves the accuarcy of the model
            bin_assignments[0].sort()
            
            training_data.append(bin_assignments[0])
            training_label.append(i)
     
    path = path.rsplit('\\',1)[0]

0
1
2
3
4
5
6
7
8
9


In [57]:
# we check if any image in our dataset have less or more than 15 descriptors
# mainly we check if they have less than 15 (as the SIFT algorithm generate less than 15 for some images)

false_samples =0
for i in range(len(training_data)):
    if (len(training_data[i]) < 15):
        elements=[1000] * (15 - len(training_data[i]))
        training_data[i] = np.append(training_data[i], elements, 0)
    
    elif (len(training_data[i]) > 15):
        index=[range(15, len(training_data[i]),1)]
        training_data[i] = np.delete(training_data[i], index)
#     print(training_data[i])
    
    
    if(len(training_data[i]) != 15):
        false_samples += 1

print (false_samples)

0


In [58]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=15, random_state=0)

clf.fit(training_data, training_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [59]:
# here the steps of finding the bin assignments and limiting or adding psedo descriptors to the the asssignments to one step

path = ".\MNIST_Dataset_JPG_format\MNIST_JPG_testing"
test_data = []
test_labels = []

for i in range(10):
    path = path + "\\" + str(i)
    print(i)
    for f in os.listdir(path):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_images:
            continue
        image = cv2.imread(os.path.join(path,f))
        keypoints, descriptors = sift.detectAndCompute(image, None)
        if len(keypoints) != 0:
#             bin_assignments = kmeans.predict(descriptors)
            bin_assignments = kmeans_faiss.index.search(descriptors.astype(np.float32), 1)[1]
            bin_assignments = bin_assignments.reshape(1, bin_assignments.shape[0])[0]
#             print(bin_assignments[0])
            if (len(bin_assignments) < 15):
                elements=[1000] * (15 - len(bin_assignments))
                bin_assignments = np.append(bin_assignments, elements, 0)
    
            elif (len(bin_assignments) > 15):
                index=[range(15, len(bin_assignments), 1)]
                bin_assignments = np.delete(bin_assignments, index)
            bin_assignments.sort()
            test_data.append(bin_assignments)
            test_labels.append(i)
            
    path = path.rsplit('\\',1)[0]

0
1
2
3
4
5
6
7
8
9


In [60]:
wrong_len = 0
print("len of test labels: ", len(test_labels))
print("len of test data: ", len(test_data))
for sample in test_data:
    if len(sample) != 15:
        wrong_len += 1
        print(sample[0])
print(wrong_len)

len of test labels:  9708
len of test data:  9708
0


In [61]:
test_results = clf.predict(test_data)

In [62]:
print(test_results)

[0 0 0 ... 7 6 4]


In [63]:
error = 0
for i in range(len(test_results)):
    if test_results[i] != test_labels[i]:
        error += 1
print("errors:", error)
print("sample length: ", len(test_labels))
print("accuracy: ", (len(test_labels)-error)/len(test_labels)*100)

errors: 4508
sample length:  9708
accuracy:  53.56407086938607
